In [ ]:
import json
import ollama
import markdown
from datetime import datetime

datetime.datetime(2026, 2, 28, 0, 52, 13, 308612)

In [12]:

# Jira Ticket JSON

ticket = {
    "ticket_number": "OPS-1024",
    "summary": "Production API server inaccessible due to full filesystem",
    "description": "Production API server became unreachable. SSH login was slow and application containers were restarting repeatedly. Investigation showed root partition utilization at 100%, preventing services from writing logs and temporary files.",
    "incident_type": "Infrastructure",
    "priority": "Critical",
    "reporter": "Monitoring Alert - Disk Usage",
    "assignee": "DevOps Team",
    "root_cause": "Uncontrolled growth of application log files and absence of automated log rotation caused root filesystem exhaustion.",
    "solution": "Deleted large log files and temporary cache data. Configured logrotate for daily rotation with retention policy. Increased root disk volume by 20GB and added disk usage alerts at 80% and 90% thresholds."
}


In [13]:

# Prompt

prompt = f"""
You are a senior DevOps engineer.

Generate a professional troubleshooting article in clean Markdown format.

Sections required:
- Issue Summary
- Description
- Root Cause Analysis
- Troubleshooting & Resolution Steps (numbered)
- Preventive Measures

IMPORTANT:
- Use proper Markdown formatting.
- Use numbered lists for ordered steps.
- Use dash (-) for bullet points.
- Ensure proper spacing before lists.

Ticket Data:
{json.dumps(ticket, indent=2)}
"""

In [14]:
# Generate Article from Ollama

response = ollama.chat(
    model="qwen2.5:3b",
    messages=[{"role": "user", "content": prompt}]
)

article_markdown = response["message"]["content"]

article_markdown = article_markdown.replace("1.", "\n1.")
article_markdown = article_markdown.replace("- ", "\n- ")


In [16]:
# Convert Markdown → HTML

article_html_body = markdown.markdown(
    article_markdown,
    extensions=[
        "fenced_code",
        "tables",
        "sane_lists" 
    ]
)

# Wrap in Professional HTML Template

html_template = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>{ticket['ticket_number']} Troubleshooting Article</title>
<style>
body {{
    font-family: Arial, sans-serif;
    margin: 40px;
    line-height: 1.7;
    background-color: #f9f9f9;
}}
.container {{
    background: white;
    padding: 40px;
    border-radius: 8px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.05);
}}
ul, ol {{
    margin-left: 25px;
}}

li {{
    margin-bottom: 8px;
}}
h1, h2, h3 {{
    color: #2c3e50;
}}
code {{
    background-color: #f4f4f4;
    padding: 4px 6px;
    border-radius: 4px;
}}
pre {{
    background-color: #f4f4f4;
    padding: 12px;
    overflow-x: auto;
    border-radius: 6px;
}}
footer {{
    margin-top: 40px;
    font-size: 12px;
    color: gray;
}}
</style>
</head>
<body>
<div class="container">
<h1>Troubleshooting Article – {ticket['ticket_number']}</h1>
{article_html_body}
<footer>
Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
</footer>
</div>
</body>
</html>
"""

# Save HTML File
file_name = f"{ticket['ticket_number']}_Troubleshooting_Article.html"

with open(file_name, "w", encoding="utf-8") as f:
    f.write(html_template)

print(f"HTML file generated: {file_name}")

HTML file generated: OPS-1024_Troubleshooting_Article.html
